In [1]:
import pandas as pd

In [1]:
from helper_function import metadata, misc

In [8]:
data_path = "data/hospitcal_pricing/132655001_Kings County Hospital Center_standardcharges.csv.xls"

data= pd.read_csv(data_path)

/var/folders/gq/f0m_7zn13nx9vghg_5vt_c700000gp/T/ipykernel_86771/3330031807.py:3: DtypeWarning: Columns (3,10,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv(data_path)


In [9]:
data_mini = data[0:1000]

In [10]:
# List all the unique Code Type 
data["Code Type"].unique()

array(['SUP', 'ERX', 'EAP', 'DRG'], dtype=object)

**The "Code Type" in medical billing and hospital data** often refers to different coding systems or categories used to classify medical procedures, supplies, prescriptions, and diagnoses. The abbreviations 'SUP', 'ERX', 'EAP', and 'DRG' each represent a specific type of code or category. Here’s a brief explanation of each:

- **SUP (Supply):** This likely refers to codes related to medical supplies. These codes are used to identify and bill for medical supplies used during patient care, such as bandages, syringes, or medical devices.

- **ERX (Electronic Prescription):** 'ERX' codes might be related to electronic prescriptions. This could involve coding used in the electronic transmission of prescription information, often between a healthcare provider and a pharmacy.

- **EAP (Employee Assistance Program):** 'EAP' could refer to services or procedures related to Employee Assistance Programs. These programs often provide a variety of services to employees, including counseling, mental health services, and support for personal problems or work-related issues. However, it's less common to see EAP as a medical coding category, so this interpretation might vary based on specific healthcare organizational practices.

- **DRG (Diagnosis-Related Group):** DRGs are a system of classification for hospital cases. They are used to categorize patients based on diagnoses, types of procedures performed, age, gender, and other criteria. DRGs are primarily used in inpatient settings and are a critical part of the Medicare reimbursement system. They help in determining how much Medicare pays the hospital for a patient's stay.

Each of these code types serves a different purpose in the healthcare system, typically related to billing and categorization of services or products used in patient care. The specific use and meaning of these codes can vary depending on the healthcare provider's internal systems and the context in which they are used.

---

In [11]:
data.columns

Index(['Procedure', 'Code Type', 'Code', 'NDC', 'Rev Code',
       'Procedure Description', 'Payer', 'Plan(s)', 'IP Price', 'OP Price',
       'Discounted Cash Price', 'IP Negotiated Charge', 'OP Negotiated Charge',
       'PB Negotiated Charge', 'De-Identified Minimum Negotiated Charge (IP)',
       'De-Identified Maximum Negotiated Charge (IP)',
       'De-Identified Minimum Negotiated Charge (OP)',
       'De-Identified Maximum Negotiated Charge (OP)'],
      dtype='object')

In [12]:
# List all pricing columns that need to be melted
price_columns = [
    'IP Price', 'OP Price', 'Discounted Cash Price', 'IP Negotiated Charge', 
    'OP Negotiated Charge', 'PB Negotiated Charge', 
    'De-Identified Minimum Negotiated Charge (IP)', 
    'De-Identified Maximum Negotiated Charge (IP)', 
    'De-Identified Minimum Negotiated Charge (OP)', 
    'De-Identified Maximum Negotiated Charge (OP)'
]

# Map 'Price_Type' to 'rate_category' and 'setting'
def map_rate_category(price_type):
    if 'IP' in price_type:
        setting = 'inpatient'
    elif 'OP' in price_type:
        setting = 'outpatient'
    else:
        setting = 'both'  # Adjust as necessary

    if 'Discounted Cash' in price_type:
        return setting, 'cash'
    elif 'Negotiated Charge' in price_type:
        return setting, 'negotiated'
    elif 'Minimum' in price_type:
        return setting, 'min'
    elif 'Maximum' in price_type:
        return setting, 'max'
    else:
        return setting, 'gross'


# Update the mapping dictionary
mapping = {
    'Procedure Description': 'description',
    'Rev Code': 'rev_code',
    'Code': 'code',
    'NDC': 'ndc',
    'Payer': 'payer_name',
    'Plan(s)': 'plan_name',
    'Setting': 'setting',
    'Rate_Category': 'rate_category',
    'Price': 'standard_charge',
    "Procedure": "local_code"
}

# Efficient Melt Operation
melted_data = data.melt(id_vars=['Procedure', 'Code Type', 'Code', 'NDC', 'Rev Code',
                                 'Procedure Description', 'Payer', 'Plan(s)'],
                        value_vars=price_columns,
                        var_name='Price_Type',
                        value_name='Price')

# Precompute Mapping for 'Price_Type'
price_type_mapping = {price_type: map_rate_category(price_type) for price_type in price_columns}
melted_data['Setting'] = melted_data['Price_Type'].map(lambda x: price_type_mapping[x][0])
melted_data['Rate_Category'] = melted_data['Price_Type'].map(lambda x: price_type_mapping[x][1])


# TODO - Remove price type columns
melted_data.drop(columns=["Code Type", "Price_Type"], inplace=True)
melted_data.rename(columns=mapping, inplace=True)



In [ ]:
billing_code = metadata.billing_code
melted_data['billing_code'] = melted_data['code'].str.split(' ', n=1, expand=True)[0]
unique_billing_code = melted_data['billing_code'].unique()
billing_code_mapping = {code: misc.get_best_match(code, billing_code) for code in unique_billing_code}

melted_data['billing_code'] = melted_data['billing_code'].map(billing_code_mapping)

# Create indicator variables for each billing code
billing_code_dummies = pd.get_dummies(melted_data['billing_code'], prefix='Category')

# Multiply indicator variables with 'code' column
for category in billing_code_dummies.columns:
    melted_data[category] = billing_code_dummies[category] * melted_data['code']

melted_data.drop(columns=["Category_Other"], inplace=True)